# **Human Body State Monitoring Using Biometric Signals**

# *Imports*

In [1]:
!pip install torchinfo --quiet

In [2]:
import random
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import random_split
import wandb

from sklearn.metrics import accuracy_score
import gc

import glob

import zipfile
from tqdm.auto import tqdm
import os
import datetime

from sklearn.preprocessing import StandardScaler
from torch.cuda.amp import autocast, GradScaler

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)



Device:  cuda


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# *Data Preprocessing*

In [4]:
df = pd.read_csv("/content/gdrive/MyDrive/WESAD_raw_data.csv")

In [5]:
df.head()

,Time,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,...,KURT_SQUARE,HR_SQRT,MEAN_RR_MEAN_MEAN_REL_RR,SD2_LF,HR_LF,HR_HF,HF_VLF,subject id,condition,SSSQ
0,2.500754,654.010015,649.40833,49.342563,11.677158,11.676994,4.225563,92.266721,5.75,0.25,...,0.217992,9.605557,1.925085e+09,0.185010,0.246361,0.868076,0.367149,10,baseline,3
1,2.504921,654.063862,649.40833,49.300854,11.611629,11.611626,4.245817,92.258202,5.75,0.25,...,0.212375,9.605113,1.956097e+08,0.181651,0.242055,0.865946,0.377283,10,baseline,3
2,2.509087,654.024936,649.40833,49.326015,11.627159,11.627112,4.242310,92.264235,5.75,0.25,...,0.216107,9.605427,-4.897165e+06,0.178633,0.237930,0.865327,0.386985,10,baseline,3
3,2.513254,653.968183,649.40833,49.369481,11.417706,11.417571,4.323940,92.273203,5.50,0.25,...,0.222014,9.605894,-2.305165e+06,0.175917,0.234067,0.865965,0.396084,10,baseline,3
4,2.517421,654.048424,649.40833,49.279428,11.230305,11.228757,4.388076,92.259766,5.25,0.25,...,0.214546,9.605195,-2.205377e+06,0.172920,0.230420,0.867653,0.403953,10,baseline,3


In [6]:
df['condition'].unique()

array(['baseline', 'amusement', 'meditation', 'stress'], dtype=object)

In [7]:
label_map = {'baseline': 0, 'stress': 1, 'amusement': 2, 'meditation': 3}
df['label'] = df['condition'].map(label_map)

In [8]:
df['label'].unique()

array([0, 2, 3, 1])

In [9]:
drop_cols = ['condition', 'SSSQ']
df = df.drop(columns=drop_cols)

In [10]:
df['subject id'].head()

,subject id
0,10
1,10
2,10
3,10
4,10


In [11]:
feature_cols = [col for col in df.columns if col not in ['subject id', 'Time', 'label']]

In [12]:
len(feature_cols)

62

In [13]:
scaler = StandardScaler()
df[feature_cols] = scaler.fit_transform(df[feature_cols])

# Dataset Class(Including Preprocessing)

In [33]:
class WESAD(Dataset):
  def __init__(self, df, subjects_to_include=None, feature_cols=None):

    self.X_seq, self.y_seq, self.lens = [], [], []

    grouped = df.groupby('subject id')

    for subject_id, group in grouped:
      if subjects_to_include is not None and subject_id not in subjects_to_include:
        continue

      group = group.sort_values('Time')
      X = group[feature_cols].values
      y = group['label'].values[0]

      self.X_seq.append(torch.tensor(X, dtype=torch.float32))
      self.y_seq.append(y)
      self.lens.append(len(X))

    self.length = len(self.y_seq)

  def __getitem__(self, ind):
    X = self.X_seq[ind]
    y = torch.tensor(self.y_seq[ind], dtype=torch.int64)

    return X, y

  def __len__(self):
    return self.length

  def collate_fn(self, batch):
    batch_X, batch_y = zip(*batch)

    X_lens = [X.shape[0] for X in batch_X]

    batch_X = pad_sequence(batch_X, batch_first=True)

    return batch_X, torch.tensor(batch_y, dtype=torch.int64), torch.tensor(X_lens)

# *Model*

In [15]:
torch.cuda.empty_cache()

class PermuteBlock(torch.nn.Module):
  def forward(self, x):
    return x.transpose(1, 2)

## Pyramidal-Bi-LSTM

In [16]:
class pBLSTM(torch.nn.Module):
  def __init__(self, input_size, hidden_size=128):
    super(pBLSTM, self).__init__()

    self.blstm1 = nn.LSTM(input_size*2, hidden_size, batch_first=True, bidirectional=True, dropout=0.2)
    self._init_weights()

  def forward(self, x_packed):
    x_unpacked, lens_unpacked = pad_packed_sequence(x_packed, batch_first=True)

    x_reshaped, x_lens_reshaped = self.trunc_reshape(x_unpacked, lens_unpacked)

    x_packed = pack_padded_sequence(x_reshaped, x_lens_reshaped, enforce_sorted=False, batch_first=True)

    out, _ = self.blstm1(x_packed)

    return out

  def trunc_reshape(self, x, x_lens):
    T = x.shape[1]
    if T % 2 != 0:
      x = x[:, :-1, :]
      x_lens = x_lens - 1

    B, T, F = x.shape

    x = torch.reshape(x, (B, T//2, F*2))
    x_lens = torch.clamp(x_lens // 2, min=1)

    return x, x_lens

  def _init_weights(self):
    for name, param in self.blstm1.named_parameters():
      if 'weight_ih' in name:
        nn.init.xavier_uniform_(param.data)
      elif 'weight_hh' in name:
        nn.init.orthogonal_(param.data)
      elif 'bias' in name:
        param.data.fill_(0)
        n = param.size(0)
        param.data[n//4:n//2].fill_(1)


## Locked Dropout(for LSTM Layer)

In [17]:
class LockedDropout(nn.Module):
    """ LockedDropout applies the same dropout mask to every time step.

    **Thank you** to Sales Force for their initial implementation of :class:`WeightDrop`. Here is
    their `License
    <https://github.com/salesforce/awd-lstm-lm/blob/master/LICENSE>`__.

    Args:
        p (float): Probability of an element in the dropout mask to be zeroed.
    """

    def __init__(self, p=0.5):
        self.p = p
        super().__init__()

    def forward(self, x):
        """
        Args:
            x (:class:`torch.FloatTensor` [sequence length, batch size, rnn hidden size]): Input to
                apply dropout too.
        """
        if not self.training or not self.p:
            return x
        x = x.clone()
        mask = x.new_empty(1, x.size(1), x.size(2), requires_grad=False).bernoulli_(1 - self.p)
        mask = mask.div_(1 - self.p)
        mask = mask.expand_as(x)
        return x * mask


    def __repr__(self):
        return self.__class__.__name__ + '(' \
            + 'p=' + str(self.p) + ')'

## Unpack and Pack Sequences

In [18]:
class Unpack(torch.nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, x_packed):
    x_unpacked, x_lens = pad_packed_sequence(x_packed, batch_first=True)

    return x_unpacked, x_lens


In [19]:
class Pack(torch.nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, x, x_lens):
    x_packed = pack_padded_sequence(x, x_lens, enforce_sorted=False, batch_first=True)

    return x_packed

## Encoder(Embedding layer + Sequence Model)

In [20]:
class Encoder(torch.nn.Module):
  def __init__(self, input_size, encoder_hidden_size=128):
    super(Encoder, self).__init__()

    self.permute = PermuteBlock()
    self.embedding = nn.Conv1d(input_size, 128, kernel_size=3, padding=1, stride=1)

    self.pBLSTMs = torch.nn.Sequential(
        pBLSTM(128, encoder_hidden_size),
        pBLSTM(2*encoder_hidden_size, encoder_hidden_size),
    )

    self.pooling = nn.AdaptiveAvgPool1d(1)
    self.locked_dropout = LockedDropout()

    self.pack = Pack()
    self.unpack = Unpack()

    self._init_weights()

  def forward(self, x, x_lens):
    x = self.permute(x)
    x = self.embedding(x)
    x = self.permute(x)

    for layer in self.pBLSTMs:
      x = self.pack(x, x_lens)
      x = layer(x)
      x, x_lens = self.unpack(x)
      x = self.permute(x)
      x = self.locked_dropout(x)
      x = self.permute(x)

    encoder_outputs, encoder_lens = (x, x_lens)

    encoder_outputs = self.permute(encoder_outputs)
    encoder_outputs = self.pooling(encoder_outputs)
    encoder_outputs = encoder_outputs.squeeze(-1)

    return encoder_outputs, encoder_lens

  def _init_weights(self):
    for m in self.modules():
      if isinstance(m, nn.Conv1d):
        nn.init.xavier_normal_(m.weight)


## Decoder(Classifier)

In [21]:
class Decoder(torch.nn.Module):
  def __init__(self, embed_size, output_size=4):
    super().__init__()
    self.mlp = nn.Sequential(
        nn.Linear(2*embed_size, 128),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(128, output_size)
      )



    self._init_weights()

  def forward(self, encoder_out):
    out = self.mlp(encoder_out)

    return out

  def _init_weights(self):
    for m in self.modules():
      if isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight)


## Final Model(Encoder + Decoder)

In [22]:
class StressModel(torch.nn.Module):
  def __init__(self, input_size, embed_size=128, output_size=4):
    super().__init__()

    self.encoder = Encoder(input_size, embed_size)
    self.decoder = Decoder(embed_size, output_size)

  def forward(self, x, x_lens):
    encoder_out, encoder_lens = self.encoder(x, x_lens)

    decoder_out = self.decoder(encoder_out)

    return decoder_out, encoder_lens


In [23]:
IN_SIZE = len(feature_cols)
EMBED_SIZE = 128
OUT_SIZE = 4

In [24]:
model = StressModel(input_size=IN_SIZE, embed_size=EMBED_SIZE, output_size=OUT_SIZE).to(device)
print(model)

StressModel(
  (encoder): Encoder(
    (permute): PermuteBlock()
    (embedding): Conv1d(62, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (pBLSTMs): Sequential(
      (0): pBLSTM(
        (blstm1): LSTM(256, 128, batch_first=True, dropout=0.2, bidirectional=True)
      )
      (1): pBLSTM(
        (blstm1): LSTM(512, 128, batch_first=True, dropout=0.2, bidirectional=True)
      )
    )
    (pooling): AdaptiveAvgPool1d(output_size=1)
    (locked_dropout): LockedDropout(p=0.5)
    (pack): Pack()
    (unpack): Unpack()
  )
  (decoder): Decoder(
    (mlp): Sequential(
      (0): Linear(in_features=256, out_features=128, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.3, inplace=False)
      (3): Linear(in_features=128, out_features=4, bias=True)
    )
  )
)


In [25]:
B, T, F = 2, 300, 62
dummy_x = torch.randn(B, T, F).to(device)
dummy_x_lens = torch.tensor([T, T-10])


summary(model, input_data=(dummy_x, dummy_x_lens), depth=3, col_names=["input_size", "output_size", "num_params"])


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
StressModel                              [2, 300, 62]              [2, 4]                    --
├─Encoder: 1-1                           [2, 300, 62]              [2, 256]                  --
│    └─PermuteBlock: 2-1                 [2, 300, 62]              [2, 62, 300]              --
│    └─Conv1d: 2-2                       [2, 62, 300]              [2, 128, 300]             23,936
│    └─PermuteBlock: 2-3                 [2, 128, 300]             [2, 300, 128]             --
│    └─Pack: 2-4                         [2, 300, 128]             [590, 128]                --
│    └─Sequential: 2-11                  --                        --                        (recursive)
│    │    └─pBLSTM: 3-1                  [590, 128]                [295, 256]                395,264
│    └─Unpack: 2-6                       [295, 256]                [2, 150, 256]             --
│    └─PermuteBlo

# *Train and Eval Functions*

In [26]:
def train_one_epoch(model, loader, criterion, optimizer, scaler):
    model.train()
    total_loss = 0
    for x, y, lens in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            out, _ = model(x, lens)
            loss = criterion(out, y)

        total_loss += loss.item()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

    return total_loss / len(loader)


In [27]:
def evaluate(model, loader):
    model.eval()
    y_true, y_pred = [], []
    total_loss = 0

    with torch.no_grad():
        for x, y, lens in loader:
            x, y = x.to(device), y.to(device)
            out, _ = model(x, lens)

            pred = torch.argmax(out, dim=1)
            y_true.extend(y.cpu().numpy())
            y_pred.extend(pred.cpu().numpy())

    acc = accuracy_score(y_true, y_pred)
    f1  = f1_score(y_true, y_pred, average='weighted')
    return 0, acc, f1


# Wandb for Metric Logging

In [28]:
import wandb
wandb.login(key="39e9c89279f6d046c7bae725e099c70ddf0fd98f")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rishitsaxena55 (rishitsaxena55-indian-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [29]:
run = wandb.init(
    name = "first", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    #id = "rbborunl", ### Insert specific run id here if you want to resume a previous run
    #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "Body State Monitoring", ### Project should be created in your wandb account
)

wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


In [30]:
# This is for checkpointing, if you're doing it over multiple sessions

# last_epoch_completed = epoch
# start = last_epoch_completed + 1
# end = config["finetune_epochs"]
# best_lev_dist = metric # if you're restarting from some checkpoint, use what you saw there.
# epoch_model_path = "/content/gdrive/MyDrive"  # set the model path( Optional, you can just store best one. Make sure to make the changes below )
best_model_path = "/content/gdrive/MyDrive" # set best model path

# *LOSO(Leave-One-Subject-Out) Training Loop*

In [36]:
import wandb
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score

# Preprocessed df and feature_cols must be defined outside
all_subjects = df['subject id'].unique()
results = {}

for test_subject in all_subjects:
    print(f"\n Testing on subject {test_subject} (LOSO)")

    wandb.init(project="wesad-loso", name=f"Subject_{test_subject}", reinit=True)

    # Split data for LOSO
    train_dataset = WESAD(df, subjects_to_include=[s for s in all_subjects if s != test_subject], feature_cols=feature_cols)
    test_dataset  = WESAD(df, subjects_to_include=[test_subject], feature_cols=feature_cols)

    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=train_dataset.collate_fn)
    test_loader  = DataLoader(test_dataset,  batch_size=1, shuffle=False, collate_fn=test_dataset.collate_fn)

    # Model setup
    model = StressModel(input_size=train_dataset.X_seq[0].shape[1], output_size=4).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    scaler = torch.cuda.amp.GradScaler()

    best_f1 = 0

    for epoch in range(10):
        print(f"Epoch {epoch+1}/10")
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, scaler)
        test_loss, acc, f1 = evaluate(model, test_loader)  # Assuming your evaluate returns these 3

        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "val_loss": test_loss,
            "val_acc": acc,
            "val_f1": f1,
            "lr": optimizer.param_groups[0]['lr']
        })

        # Save best model
        if f1 > best_f1:
            best_f1 = f1
            torch.save(model.state_dict(), os.path.join(best_model_path, f"best_subject_{test_subject}.pt"))

    wandb.finish()

    print(f" Subject {test_subject} — Acc: {acc:.4f}, F1: {f1:.4f}")
    results[test_subject] = {"acc": acc, "f1": f1}



🔁 Testing on subject 10 (LOSO)


epoch,▁
lr,▁
train_loss,▁
val_acc,▁
val_f1,▁
val_loss,▁
epoch,1
lr,0.001
train_loss,0.72028
val_acc,1
val_f1,1


Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10


epoch,▁▂▃▃▄▅▆▆▇█
lr,▁▁▁▁▁▁▁▁▁▁
train_loss,█▂▁▁▁▁▁▁▁▁
val_acc,▁▁▁▁▁▁▁▁▁▁
val_f1,▁▁▁▁▁▁▁▁▁▁
val_loss,▁▁▁▁▁▁▁▁▁▁
epoch,10
lr,0.001
train_loss,0.00011
val_acc,1
val_f1,1


✅ Subject 10 — Acc: 1.0000, F1: 1.0000

🔁 Testing on subject 9 (LOSO)


...

epoch,▁▂▃▃▄▅▆▆▇█
lr,▁▁▁▁▁▁▁▁▁▁
train_loss,█▂▁▁▁▁▁▁▁▁
val_acc,▁▁▁▁▁▁▁▁▁▁
val_f1,▁▁▁▁▁▁▁▁▁▁
val_loss,▁▁▁▁▁▁▁▁▁▁
epoch,10
lr,0.001
train_loss,0.00036
val_acc,1
val_f1,1


✅ Subject 15 — Acc: 1.0000, F1: 1.0000

🔁 Testing on subject 7 (LOSO)


Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10


epoch,▁▂▃▃▄▅▆▆▇█
lr,▁▁▁▁▁▁▁▁▁▁
train_loss,█▃▁▁▁▁▁▁▁▁
val_acc,▁▁▁▁▁▁▁▁▁▁
val_f1,▁▁▁▁▁▁▁▁▁▁
val_loss,▁▁▁▁▁▁▁▁▁▁
epoch,10
lr,0.001
train_loss,0.00028
val_acc,1
val_f1,1


✅ Subject 7 — Acc: 1.0000, F1: 1.0000
